<a href="https://colab.research.google.com/github/wbandabarragan/EPIC_3/blob/main/EPIC_Junior/evento-septiembre-2023/2_Atomo_Hidrogeno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hydrogen atom orbitals

Original source: https://github.com/sebastianmagns/hydrogen-wavefunctions

In [1]:
#!pip install seaborn


In [2]:
from scipy.constants import physical_constants
import matplotlib.pyplot as plt
import scipy.special as sp
import seaborn as sns
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact


In [3]:
def radial_function(n, l, r, a0):
    """ Calcular la parte radial, i.e. el tamaño de la onda de probabilidad.

    Argumentos:
        n (número entero): número asociado al nivel de energía
        l (número entero): número asociado a la forma de los orbitales

        r (numpy.ndarray): coordenada radial
        a0 (número decimal): radio de Bohr

    Resultato:
        numpy.ndarray: la componente radial de la onda
    """

    laguerre = sp.genlaguerre(n - l - 1, 2 * l + 1)
    p = 2 * r / (n * a0)

    constant_factor = np.sqrt(
        ((2 / n * a0) ** 3 * (sp.factorial(n - l - 1))) /
        (2 * n * (sp.factorial(n + l)))
    )
    return constant_factor * np.exp(-p / 2) * (p ** l) * laguerre(p)



In [4]:
def angular_function(m, l, theta, phi):
    """
    Calcula la parte angular de la funcion de onda, usando polinomios de Legendre,
    (se encuentra en tablas, no hay que saberlos de memoria).

    Argumentos:
        m (número entero): número asociado a la magnetización del átomo, es decir la orientación
        l (número entero): número asociado a la forma de los orbitales
        theta (numpy.ndarray): ángulo polar
        phi (int): ángulo azimutal

    Resultado:
        numpy.ndarray: la componente angular de la onda
    """

    legendre = sp.lpmv(m, l, np.cos(theta))

    constant_factor = ((-1) ** m) * np.sqrt(
        ((2 * l + 1) * sp.factorial(l - np.abs(m))) /
        (4 * np.pi * sp.factorial(l + np.abs(m)))
    )
    return constant_factor * legendre * np.real(np.exp(1.j * m * phi))




In [5]:
def compute_wavefunction(n, l, m, a0_scale_factor):

    """
    Calcula la función de onda total como producto de las dos funciones de onda
    definidas en las dos celdas anteriores.

    Argumentos:

        n (número entero): número asociado al nivel de energía
        l (número entero): número asociado a la forma de los orbitales
        m (número entero): número asociado a la magnetización del átomo, es decir la orientación
        a0_scale_factor (número decimal): Factor de escala para el radio de Bohr
    Resultado:
        numpy.ndarray: función de onda general
    """

    # Scale Bohr radius for effective visualization
    a0 = a0_scale_factor * physical_constants['Bohr radius'][0] * 1e+12

    # x-y grid to represent electron spatial distribution
    grid_extent = 480
    grid_resolution = 680
    x = y = np.linspace(-grid_extent, grid_extent, grid_resolution)
    x, y = np.meshgrid(x, y)

    # Use epsilon to avoid division by zero during angle calculations
    eps = np.finfo(float).eps

    # Ψnlm(r,θ,φ) = Rnl(r).Ylm(θ,φ)
    psi = radial_function(
        n, l, np.sqrt((x ** 2 + y ** 2)), a0
    ) * angular_function(
        m, l, np.arctan(x / (y + eps)), 0
    )
    return psi


In [6]:
def compute_probability_density(psi):
    """
    Calcula la probabilidad en base de la función de onda

    Argumentos:
        psi (numpy.ndarray): función de onda

    Resultado:
        numpy.ndarray: la distribución de probabilidades
    """
    return np.abs(psi) ** 2



In [7]:
def plot_wf_probability_density(n, l, m, a0_scale_factor, dark_theme=False, color_palette='rocket'):

    """
    Gráfico de la distribución del átomo de Hidrógeno
    para valores dados de (n,l,m)

    Argumentos:
        n (número entero): número asociado al nivel de energía
        l (número entero): número asociado a la forma de los orbitales
        m (número entero): número asociado a la magnetización del átomo, es decir la orientación
        a0_scale_factor (número decimal): Factor de escala para el radio de Bohr

        dark_theme (bool): If True, uses a dark background for the plot, defaults to False
        color_palette (str): Seaborn plot color palette, defaults to 'rocket'
    """

    # Quantum numbers validation
    if not isinstance(n, int) or n < 1:
        raise ValueError('n should be an integer satisfying the condition: n >= 1')
    if not isinstance(l, int) or not (0 <= l < n):
        raise ValueError('l should be an integer satisfying the condition: 0 <= l < n')
    if not isinstance(m, int) or not (-l <= m <= l):
        raise ValueError('m should be an integer satisfying the condition: -l <= m <= l')

    # Color palette validation
    try:
        sns.color_palette(color_palette)
    except ValueError:
        raise ValueError(f'{color_palette} is not a recognized Seaborn color palette.')

    fig, ax = plt.subplots(figsize=(8, 8.5))
    plt.subplots_adjust(top=0.82)
    plt.subplots_adjust(right=0.905)
    plt.subplots_adjust(left=-0.1)

    # Compute and visualize the wavefunction probability density
    psi = compute_wavefunction(n, l, m, a0_scale_factor)
    prob_density = compute_probability_density(psi)
    im = ax.imshow(np.sqrt(prob_density), cmap=sns.color_palette(color_palette, as_cmap=True))

    cbar = plt.colorbar(im, fraction=0.046, pad=0.03)
    cbar.set_ticks([])

    # Apply dark theme parameters
    if dark_theme:
        theme = 'dt'
        background_color = sorted(
            sns.color_palette(color_palette, n_colors=100),
            key=lambda color: 0.2126 * color[0] + 0.7152 * color[1] + 0.0722 * color[2]
        )[0]
        plt.rcParams['text.color'] = '#dfdfdf'
        title_color = '#dfdfdf'
        fig.patch.set_facecolor(background_color)
        cbar.outline.set_visible(False)
        ax.tick_params(axis='x', colors='#c4c4c4')
        ax.tick_params(axis='y', colors='#c4c4c4')
        for spine in ax.spines.values():
            spine.set_color('#c4c4c4')

    else:  # Apply light theme parameters
        theme = 'lt'
        plt.rcParams['text.color'] = '#000000'
        title_color = '#000000'
        ax.tick_params(axis='x', colors='#000000')
        ax.tick_params(axis='y', colors='#000000')

    ax.text(30, 615, r'$({0}, {1}, {2})$'.format(n, l, m), color='#dfdfdf', fontsize=22)
    ax.text(705, 700, 'Muy\nprobable', fontsize=12)
    ax.text(705, -60, 'Poco\nprobable', fontsize=12)
    ax.invert_yaxis()

    # Save and display the plot
    plt.savefig(f'({n},{l},{m})[{theme}].png')
    plt.show()


# Imagen interactiva

In [8]:
n_value = 9
n=n_value


In [9]:
# Función interactiva para actualizar los plots en tiempo real

def interactive_plot(n, l, m):
    a0_scale_factor = 1 / n  # Calcula el a0_scale_factor en función de n

    plot = plot_wf_probability_density(n, l, m, a0_scale_factor, dark_theme=False, color_palette='rocket')
    plt.show()

# Crea controles deslizantes para n, l, y m
n_slider = widgets.IntSlider(value=n, min=1, max=n_value, description='n')
l_slider = widgets.IntSlider(value=0, min=0, max=n_slider.value-1, description='l')
m_slider = widgets.IntSlider(value=0, min=-l_slider.value, max=l_slider.value, description='m')

# Define función para actualizar l_slider cuando cambia n_slider
def update_l_range(change):
    l_slider.max = change.new - 1
    l_slider.value = max(l_slider.value, 0)  # Asegura que el valor de l se mantenga dentro del nuevo rango
    update_m_range(None)  # Actualiza el rango de m al cambiar l

# Vincula la función de actualización a cambios en n_slider
n_slider.observe(update_l_range, names='value')

# Define función para actualizar m_slider cuando cambia l_slider
def update_m_range(change):
    m_slider.min = -l_slider.value  # Ajusta el valor mínimo de m en función de l
    m_slider.max = l_slider.value
    m_slider.value = max(min(m_slider.value, m_slider.max), m_slider.min)  # Asegura que el valor de m esté dentro del rango permitido

# Vincula la función de actualización a cambios en l_slider
l_slider.observe(update_m_range, names='value')

# Crea la interfaz interactiva utilizando la función interact
interact(interactive_plot, n=n_slider, l=l_slider, m=m_slider)


interactive(children=(IntSlider(value=9, description='n', max=9, min=1), IntSlider(value=0, description='l', m…

<function __main__.interactive_plot(n, l, m)>